# Post processing and plotting
This notebook requires the `.bz2` file created in the previous notebook `01-ExampleScripts\01-RAWdata_to_Results.ipynb`, by analyzing the folder `02-ExampleData\PDMS_phantom`, with the analysis setting `save_zip` set to `True`.

In [ ]:
## imports
import os 
import sys
import numpy as np
import cv2 as cv

from skimage import data, img_as_float
from skimage.segmentation import (morphological_chan_vese, checkerboard_level_set)
from scipy.stats import skew

import bz2 # to zip the pickle file: 
import pickle # to read the OCT_lib.bscan object

sys.path.insert(0,'../')
import OCT_lib

%matplotlib nbagg
import matplotlib.pyplot as plt

### Reading in the results of the previous notebook
The following cell reads the bz2 file that was created during the analysis in the example notebook `01-RAWdata_to_results` on the `02-ExampleData\PDMS_phantom` folder

### IMPORTANT NOTE
When running the following cell, the file selector prompt is sometimes **hidden** below the browser. If nothing seems to happen, please look under its window.

In [ ]:
basepath = r"..//02-ExampleData" 
data, basepath = OCT_lib.get_results(initialdir=basepath)

folder_name = data.pop('name')
p = data.pop('params') # p is the dict with all previous analysis settings
data = data.pop('results') # this is where the image and the profile of each experiment are stored
experiments = list(data.keys()) # names of all files

## A HW timing inconsistency results in the image sometimes having an extra A-scan. 
## let's clean that:
lengths = [len(data[experiment]['profile']) for experiment in experiments]
min_len = min(lengths)
for experiment in experiments:
    if len(data[experiment]['profile']) > min_len:
        data[experiment]['profile'] = data[experiment]['profile'][:min_len]

### Conversions px to real distance
The following cell creates:
 - `hor` the lost of x positions along the b-scan
 - `px_to_d_vert` the Optical-Path-difference of a pixel, corresponds to the physical height if index of refraction == 1

In [ ]:
# the physical width (mm) is divided by the number of columns (A-scans) that it takes to image it
width_Bscan = p['OCT']['aperture_size'] # assuming the BSCAN acquired at the max width of the aperture
px_to_d_hor = width_Bscan/len(data[experiments[0]]["profile"]) 
hor = np.squeeze(np.arange(0,p['OCT']['aperture_size'],px_to_d_hor))

# the axial resolution of the OCT is 5.75 um/px for an ascanlength of 1024 elements
# The following expression also considers the case in which the image has been resized
px_to_d_vert = p['OCT']['axial_res']/(p['resize']['height_factor'] if p['resize']['switch'] else 1)

## Profile height
The following cell creates the dictionary `deltaZ` that stores all the profile heights 
for each measurement (`reps`), for each condition (`rel` relaxed, or `suc` under suction), and for each region (`thin`, `thick`, `boundary`)

In [ ]:
# for this particular set of measurements on PDMS
regions = ['thin', 'thick', 'boundary']
# repeated measurements
reps = ['001', '002', '003', '004', '005']
N_locs = len(regions) # locations

# `deltaZ` blueprint -> profile[region][rep]
deltaZ = {}
for i, meas in enumerate(data.keys()): # iterate over all meeasurements 
    for region in regions: # iterate over the three regions
        deltaZ[region]=[]
        for rep in reps: # iterate over the repeated 
            name_rel = region+'_rel_'+rep
            name_suc = f"{region}_suc_{rep}"
            deltaZ[region].append(
                np.squeeze(np.array(
                    ((data[name_suc]["profile"])-(data[name_suc]["profile"][0])) -
                    ((data[name_rel]["profile"])-(data[name_rel]["profile"][0]))                    
                )*-px_to_d_vert)
            )

In [ ]:
# used for Asymmetry Factor calculations
peak_cut = 0.2 # fraction of the profile height that is taken into consideration for AF calcs

for region in regions:
    for i,rep in enumerate(reps):
        name_rel = f"{region}_rel_{rep}"
        name_suc = f"{region}_suc_{rep}"
        height, width = data[name_rel]['image'].shape
        
        delta_z = deltaZ[region][i]
        delta_z_sm = OCT_lib.smooth_profile(delta_z)
        x_range = OCT_lib.center_profile(hor, delta_z_sm)
        
        peak_z = max(delta_z_sm)
        peak_idx = int(np.argwhere(delta_z_sm == peak_z)[0])
        peak_left_idx  = int(np.argwhere(delta_z_sm > peak_z*peak_cut)[0]) 
        peak_right_idx = int(np.argwhere(delta_z_sm > peak_z*peak_cut)[-1]) 
        peak_base_middle_idx = int((peak_right_idx+peak_left_idx)/2) #used for AF3
        
        fig, ax = plt.subplots(nrows=2, ncols=2)

        ax[0,0].imshow(data[name_rel]['image'], cmap='gray', extent=[0, width, 0, height])
        ax[0,0].axis('off')
        ax[0,0].plot(height-data[name_rel]['profile'], color='red', alpha=0.5)
        # ax[0,0].scatter(peak_x, height-data[name_rel]['profile'][peak_x], s = 7, c='forestgreen', alpha=0.7, zorder = 200 )
        ax[0,0].set_title(f"Relaxed")
        
        ax[0,1].imshow(data[name_suc]['image'], cmap='gray', extent=[0, width, 0, height])
        ax[0,1].axis('off')
        ax[0,1].plot(height-data[name_suc]['profile'], color='red', alpha=0.5)
        # ax[0,1].scatter(peak_x, height-data[name_suc]['profile'][peak_x], s = 7, c='forestgreen', alpha=0.7, zorder = 200 ) 
        ax[0,1].set_title(f"Suction = 500mbar")
        
        ax[1,0].plot(x_range, delta_z, alpha=0.5, c = 'midnightblue', label = "profile")
        ax[1,0].plot(x_range, delta_z_sm, zorder =100, c = 'red', linewidth = 1.7, label="smoothed")
        ax[1,0].set_xlim(left = -p['OCT']['aperture_size']/1.85, right = p['OCT']['aperture_size']/1.85)
#         ax[1,0].axvline(x=0, c='grey', alpha =0.5)
#         ax[1,0].axhline(y = max(delta_z_sm)*peak_cut, c='grey', alpha = 0.5)
        ax[1,0].fill_between(
            x = x_range[peak_base_middle_idx:peak_right_idx], 
            y2 = delta_z_sm[peak_base_middle_idx:peak_right_idx], 
            ## common
            y1 = max(delta_z_sm)*peak_cut,
            facecolor = "forestgreen",
            alpha = 0.3)
        ax[1,0].fill_between(
            x = x_range[peak_left_idx:peak_base_middle_idx], 
            y2 = delta_z_sm[peak_left_idx:peak_base_middle_idx],
            ## common
            y1 = max(delta_z_sm)*peak_cut,
            facecolor = "red",
            alpha = 0.3)
        ax[1,0].set(ylabel='$\Delta$z ($\mu$m)')
        ax[1,0].set(xlabel='Position with respect to the peak (mm)')
#         ax[1,0].legend()
        ax[1,0].spines['bottom'].set_smart_bounds(True)
        ax[1,0].spines['left'].set_smart_bounds(True)
        ax[1,0].spines['top'].set_color('none')
        ax[1,0].spines['right'].set_color('none')
#         ax[1,0].spines['bottom'].set_position(('axes', -0.05))
        
    
        af = OCT_lib.AF(delta_z_sm, peak_cutoff = peak_cut)
        info = ''\
        f"Max Delta Z = {max(delta_z_sm):.4}\n"\
        f"Asymm. factor = {af:.2}\n"\
#         f"Skewness = {skew(delta_z_sm):.4}\n"
        ax[1,1].axis('off')
        ax[1,1].annotate(info,(0.1, 0.5),xycoords='axes fraction', va='center')
        
        fig.tight_layout()
        
        sup_title = f"B-scans and deformation measurement on '{region}-{rep}'"
        fig.suptitle(sup_title, fontsize=14)
        fig.subplots_adjust(top=0.93)
        
        figname = os.path.join(basepath, 'processed', f"{folder_name}_{region}{rep}_deltaZ.png")
        
        fig.savefig(fname = figname, dpi = 450)
        plt.close(fig)      
